In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd  
import seaborn as sns
from pmdarima import auto_arima
from deep_translator import GoogleTranslator
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# #Set working directory
# os.chdir('/Users/tejveero/Library/CloudStorage/GoogleDrive-tejveerobr@gmail.com/My Drive/SCM.800/Wireframe')
# file_path = 'model_results.xlsx'
# file_path2 = 'tableau_ingestion.xlsx'

# # Use the read_csv function to load the CSV file into a DataFrame
# df_c = pd.read_excel(file_path, sheet_name='Customers_Served')
# df_m = pd.read_excel(file_path, sheet_name='Materials_Stored')

In [2]:
# Set working directory
os.chdir('/Users/tejveero/Library/CloudStorage/GoogleDrive-tejveerobr@gmail.com/My Drive/SCM.800/Wireframe')

# Mapping of filenames to scenario labels
files = {
    'model_baseline_results.xlsx': 'Scenario 0 - Baseline',
    'model_optimal_results.xlsx': 'Scenario 1 - Optimal',
    'model_BigCust_results.xlsx': 'Scenario 2 - Large Customers',
    'model_VaR18_results.xlsx': 'Scenario 3 - Value-at-Risk',
}

# Empty lists to hold intermediate DataFrames
df_c_list = []
df_m_list = []

# Loop over files and append DataFrames with scenario column
for filename, scenario in files.items():
    temp_c = pd.read_excel(filename, sheet_name='Customers_Served')
    temp_c['Scenario'] = scenario
    df_c_list.append(temp_c)

    temp_m = pd.read_excel(filename, sheet_name='Materials_Stored')
    temp_m['Scenario'] = scenario
    df_m_list.append(temp_m)

# Combine all into single DataFrames
df_c = pd.concat(df_c_list, ignore_index=True)
df_m = pd.concat(df_m_list, ignore_index=True)

Rename fields

In [5]:
# Step 1: Rename columns in df_c
df_c = df_c.rename(columns={
    'zipcode': 'zip_code',
    'Warehouse': 'DC_to_serve_from',
    't_cost': 'lane_cost_estimate',
    'emissions': 'lane_emission_estimate'
})

# Step 2: Rename columns in df_m
df_m = df_m.rename(columns={
    'key': 'product_location_customer_code(id)',
    'Warehouse': 'target_location',
    'Material': 'product_id',
    'material_name': 'product_name',
    'rev_class': 'product_revenue_class',
    'med_class': 'product_medical_class',
    'cogs': 'product_value',
    'demand_served': 'target_inventory_level'
})

In [6]:
df_c['customer_name'].nunique()

1663

Aggregate Ambient & Refrigerated

In [7]:
##FOR DF_M
# Remove _A and _R suffixes
df_m['target_location'] = df_m['target_location'].str.replace(r'_[AR]$', '', regex=True)

# Group and aggregate
df_m = df_m.groupby(
    ['Scenario','customer_id', 'product_id', 'product_name', 'target_location', 
     'product_location_customer_code(id)', 'product_revenue_class', 'product_medical_class'],
    as_index=False
).agg({
    'target_inventory_level': 'sum',
    'product_value': 'sum'
})

In [8]:
##FOR DF_C
#Remove _A and _R suffixes
df_c['DC_to_serve_from'] = df_c['DC_to_serve_from'].str.replace(r'_[AR]$', '', regex=True)

# Step 2: Group and sum
df_c = df_c.groupby(
    ['Scenario','customer_id', 'customer_name', 'DC_to_serve_from', 'zip_code'],
    as_index=False
).agg({
    'lane_emission_estimate': 'sum',
    'lane_cost_estimate': 'sum'
})

In [9]:
df_c['customer_name'].nunique()

1663

Export to excel in drive

In [12]:
# Export df_c and df_m to an Excel file with two sheets
with pd.ExcelWriter('tableau_ingestion.xlsx') as writer:
    df_c.to_excel(writer, sheet_name='df_c', index=False)
    df_m.to_excel(writer, sheet_name='df_m', index=False)